Load flight data

In [4]:
import pandas as pd

flight_data = pd.read_csv(r'C:\Users\Gareth.Ahern\Desktop\NonCommercialOver1000\Attempt2\14.csv')
flight_data.head()

,Flight_ID,Position_DateTime,Latitude,Longitude
0,619631624,14/07/2020 09:27,51.62368,-0.31448
1,619631624,14/07/2020 09:27,51.62550,-0.30998
2,619631624,14/07/2020 09:26,51.62671,-0.35858
3,619631624,14/07/2020 09:26,51.62401,-0.34622
4,619631624,14/07/2020 09:26,51.62159,-0.33768


Edit flight data so it has a geometry column

In [5]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(flight_data.Longitude, flight_data.Latitude)]
gdf = gpd.GeoDataFrame(flight_data.drop(['Longitude', 'Latitude'], axis=1), crs={'init': 'epsg:4326'}, geometry=geometry)
gdf.head()

,Flight_ID,Position_DateTime,geometry
0,619631624,14/07/2020 09:27,POINT (-0.31448 51.62368)
1,619631624,14/07/2020 09:27,POINT (-0.30998 51.62550)
2,619631624,14/07/2020 09:26,POINT (-0.35858 51.62671)
3,619631624,14/07/2020 09:26,POINT (-0.34622 51.62401)
4,619631624,14/07/2020 09:26,POINT (-0.33768 51.62159)


In [6]:
import geopandas as gpd

shp_map = gpd.read_file(r'C:\Users\Gareth.Ahern\Desktop\gadm36_GBR_shp\gadm36_GBR_3.shp')
shp_map.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,GID_3,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry
0,GBR,United Kingdom,GBR.1_1,England,None,GBR.1.1_1,Barnsley,None,GBR.1.1.1_1,Barnsley,None,None,Metropolitan borough,Metropolitan borough,None,GB.BX.BX,"POLYGON ((-1.78997 53.47293, -1.79306 53.47778..."
1,GBR,United Kingdom,GBR.1_1,England,None,GBR.1.2_1,Bath and North East Somerset,None,GBR.1.2.1_1,Bath and North East Somerset,None,None,Unitary authority,Unitary authority,None,GB.BN.BN,"POLYGON ((-2.68629 51.31516, -2.68066 51.32836..."
2,GBR,United Kingdom,GBR.1_1,England,None,GBR.1.3_1,Bedfordshire,None,GBR.1.3.1_1,Bedford,None,None,Unitary authority,Unitary authority,None,GB.FO.FO,"POLYGON ((-0.58508 52.11364, -0.58806 52.11805..."
3,GBR,United Kingdom,GBR.1_1,England,None,GBR.1.4_1,Birmingham,None,GBR.1.4.1_1,Birmingham,None,None,Metropolitan borough (city),Metropolitan borough (city),None,GB.BI.BI,"POLYGON ((-1.73417 52.51014, -1.73984 52.50894..."
4,GBR,United Kingdom,GBR.1_1,England,None,GBR.1.5_1,Blackburn with Darwen,None,GBR.1.5.1_1,Blackburn with Darwen,None,None,Unitary authority,Unitary authority,None,GB.BW.BW,"POLYGON ((-2.51111 53.63377, -2.51317 53.63582..."


In [7]:
data_with_counties = gpd.sjoin(gdf, shp_map, how="inner", op='intersects')
data_with_counties['District'] = data_with_counties['NAME_3']
print(type(data_with_counties))

data_with_counties = data_with_counties[['Flight_ID', 'District']]
data_with_counties = data_with_counties.groupby(['District']).count()
data_with_counties.reset_index(inplace=True)
data_with_counties.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,District,Flight_ID
0,Aberdeen,675
1,Aberdeenshire,3066
2,Adur,21
3,Allerdale,52
4,Amber Valley,135


In [10]:
import folium
m = folium.Map(location=[51.157925, -0.163917], zoom_start=6)

folium.Choropleth(
    geo_data=street_map,
    name="choropleth",
    data=data_with_counties,
    columns=["District", "Flight_ID"],
    key_on="feature.properties.ctyua16nm",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="No Of Flight Signals",
).add_to(m)

folium.LayerControl().add_to(m)

#m